In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

###Combining Filters
Like factors, filters can be combined. Combining filters is done using the `&` (and) and `|` (or) operators. For example, let's say we want to screen for securities that are in the top 5% of average dollar volume and have a latest close price above $90. To start, let's make a high dollar volume filter using an `AverageDollarVolume` factor and `percentile_between`:

In [2]:
dollar_volume = AverageDollarVolume(window_length=60)
high_dollar_volume = dollar_volume.percentile_between(95, 100)

Note: `percentile_between` is a `Factor` method returning a `Filter`.

Next, let's create a `latest_close` factor and define a filter for securities that closed above $90:

In [3]:
latest_close = USEquityPricing.close.latest
above_90 = latest_close > 90

Now we can combine our `high_dollar_volume` filter with our `above_90` filter using the `&` operator:

In [4]:
tradeable_filter = high_dollar_volume & above_90

This filter will evaluate to `True` for securities where both `high_dollar_volume` and `above_20` are `True`. Otherwise, it will evaluate to `False`. A similar computation can be made with the `|` (or) operator.

If we want to use this filter as a screen in our pipeline, we can simply pass `tradeable_filter` as the `screen` argument.

In [12]:
def make_pipeline():

    mean_close_60 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=60)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_60 - mean_close_30) / mean_close_60

    dollar_volume = AverageDollarVolume(window_length=60)
    high_dollar_volume = dollar_volume.percentile_between(95, 100)

    latest_close = USEquityPricing.close.latest
    above_90 = latest_close > 90

    tradeable_filter = high_dollar_volume & above_90

    return Pipeline(
        columns = {'percent_difference': percent_difference, 'close': USEquityPricing.close.latest},
        screen = tradeable_filter)

When we run this, our pipeline output now only includes ~180 securities.

In [13]:
result = run_pipeline(make_pipeline(), '2017-12-27', '2017-12-27')
print 'Number of securities that passed the filter: %d' % len(result)
result.head(10)

Number of securities that passed the filter: 181


close  percent_difference
2017-12-27 00:00:00+00:00 Equity(24 [AAPL])   170.57           -0.032539
                          Equity(67 [ADSK])   103.80            0.025277
                          Equity(114 [ADBE])  174.46           -0.034462
                          Equity(168 [AET])   180.46           -0.038638
                          Equity(357 [TWX])    91.63            0.047407
                          Equity(368 [AMGN])  176.08            0.014175
                          Equity(438 [AON])   133.29            0.025562
                          Equity(630 [ADP])   117.58           -0.003013
                          Equity(679 [AXP])    98.57           -0.018221
                          Equity(693 [AZO])   714.29           -0.062657

In the next lesson, we'll look at masking factors and filters.